In [1]:
'''
https://www.tensorflow.org/guide/autodiff
https://www.twblogs.net/a/5b7b88322b71770a43d8bccf
自動微分法是一種介於符號微分和數值微分的方法：
數值微分強調一開始直接代入數值近似求解；
符號微分強調直接對代數進行求解，最後才代入問題數值；
自動微分將符號微分法應用於最基本的算子，
比如常數，冪函數，指數函數，對數函數，三角函數等，然後代入數值，
保留中間結果，最後再應用於整個函數。
'''
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

In [2]:
#%load_ext tensorboard

In [3]:
'''
TensorFlow "records" relevant operations executed inside 
the context of a tf.GradientTape onto a "tape". 
TensorFlow then uses that tape to compute the gradients of 
a "recorded" computation using reverse mode differentiation.
'''
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x**2

In [4]:
# dy = 2x * dx
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

6.0

In [5]:
w = tf.Variable([[1.,0.],
                 [0.,0.],
                 [0.,1.]], name='w') #(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

loss_fn_mse = tf.keras.losses.MeanSquaredError()

# 將操作tensor的運算式紀錄於GradientTape，後續可以用tape對特定運算式對特定變數計算導數
with tf.GradientTape(persistent=True) as tape:
    y = x@w + b # 應該是 tensor 內積? 維度要對
    print(y)
    loss_value = loss_fn_mse(y_true=3, y_pred=y) # mu((y_true - y_pred)^2)
    print(loss_value)

tf.Tensor([[1. 3.]], shape=(1, 2), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)


In [6]:
[dl_dw, dl_db] = tape.gradient(loss_value, [w, b])
print(w, w.shape)
print(dl_dw, dl_dw.shape)

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[1., 0.],
       [0., 0.],
       [0., 1.]], dtype=float32)> (3, 2)
tf.Tensor(
[[-2.  0.]
 [-4.  0.]
 [-6.  0.]], shape=(3, 2), dtype=float32) (3, 2)


# 畫圖

In [18]:
# 加上@tf.function裝飾器，可使返回的 tf struct 加進靜態圖，並返回導數結果

@tf.function # Compiles a function into a callable TensorFlow graph
def SetIntoGraph(x):
    return tf.matmul(x,tf.transpose(x))

In [20]:
# Set up logging.
stamp = time.strftime("%Y%m%d-%H%M%S", time.localtime())
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

w = tf.Variable([[1.,0.],
                 [0.,0.],
                 [0.,1.]], name='w') #(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

loss_fn_mse = tf.keras.losses.MeanSquaredError()

tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)
# Call only one tf.function when tracing.
z = print(SetIntoGraph(x))
with writer.as_default():
    tf.summary.trace_export(
        name="my_func_trace",
        step=0,
        profiler_outdir=logdir)

tf.Tensor([[14.]], shape=(1, 1), dtype=float32)


In [9]:
#cmd input: tensorboard --logdir logs\func